In [ ]:
from dotenv import load_dotenv
import os
import chromadb

#.env file load
load_dotenv()

HOST = os.getenv('CHROMADB_HOST')
PORT = os.getenv('CHROMADB_PORT')
COLLECTION_NAME = os.getenv('CHROMADB_COLLECTION')
UPSTAGE_KEY = os.getenv('UPSTAGE_API_KEY')
UPSTAGE_KEY_TM = os.getenv('UPSTAGE_API_KEY_TM')

client = chromadb.HttpClient(host=HOST, port=PORT)
collection = client.get_or_create_collection(COLLECTION_NAME)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
print(collection.get(include=['embeddings']))

{'ids': ['chunk_0', 'chunk_1', 'chunk_2', 'chunk_3', 'chunk_4', 'chunk_5', 'chunk_6', 'chunk_7', 'chunk_8', 'chunk_9', 'chunk_10', 'chunk_11', 'chunk_12', 'chunk_13', 'chunk_14', 'chunk_15', 'chunk_16', 'chunk_17', 'chunk_18', 'chunk_19', 'chunk_20', 'chunk_21', 'chunk_22', 'chunk_23', 'chunk_24', 'chunk_25', 'chunk_26', 'chunk_27', 'chunk_28', 'chunk_29', 'chunk_30', 'chunk_31', 'chunk_32', 'chunk_33', 'chunk_34', 'chunk_35', 'chunk_36', 'chunk_37', 'chunk_38', 'chunk_39', 'chunk_40', 'chunk_41', 'chunk_42', 'chunk_43', 'chunk_44', 'chunk_45', 'chunk_46', 'chunk_47', 'chunk_48', 'chunk_49', 'chunk_50', 'chunk_51', 'chunk_52', 'chunk_53', 'chunk_54', 'chunk_55', 'chunk_56', 'chunk_57', 'chunk_58', 'chunk_59', 'chunk_60', 'chunk_61', 'chunk_62', 'chunk_63', 'chunk_64', 'chunk_65', 'chunk_66', 'chunk_67', 'chunk_68', 'chunk_69', 'chunk_70', 'chunk_71', 'chunk_72', 'chunk_73', 'chunk_74', 'chunk_75', 'chunk_76', 'chunk_77', 'chunk_78', 'chunk_79', 'chunk_80', 'chunk_81', 'chunk_82', 'chun

In [ ]:
import re

# 텍스트 전처리 함수
def clean_text(text):
    """텍스트 전처리 개선"""
    # 페이지 번호 제거
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\s*\d+\s*\n', '', text)

    # 불필요한 특수문자 제거
    text = re.sub(r'[•◆▶▷→▪]', '', text)

    # 삭제하고 싶은 특정 문구들
    remove_patterns = [
        r'도로교통법\s*\n',
        r'.*법제처.*\n',
        r'국가법령정보센터\s*\n',
        r'confusion matrix.*?(?=\n|$)',  # confusion matrix 관련 텍스트
        r'[rR][oO][cC].*?(?=\n|$)',     # ROC 관련 텍스트
    ]

    # 모든 패턴을 하나의 정규식으로 결합
    combined_pattern = '|'.join(remove_patterns)
    text = re.sub(combined_pattern, '', text)

    # 불필요한 숫자 패턴 제거
    text = re.sub(r'^\d+\.\s*', '', text, flags=re.MULTILINE)

    # 연속된 공백 정리
    text = re.sub(r'\s+', ' ', text)

    # 불필요한 줄바꿈 정리
    text = re.sub(r'\n+', '\n', text)

    # URL 패턴 정리
    text = re.sub(r'http\S+\s?', '', text)

    return text.strip()

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2', # 한국어 자연어 추론에 최적화된 모델
        model_kwargs={'device':'cpu'}, # 모델이 CPU에서 실행되도록 설정 GPU를 사용할 수 있다면 cuda로 설정
        encode_kwargs={'normalize_embeddings':True}, # 임베딩을 정규화하여 모든 벡터가 같은 범위 값을 갖도록 한다.
    )

/var/folders/sn/nzmg6wr50bq9gv_8cx8tsqcc0000gn/T/ipykernel_15681/1773147662.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/Users/antaemin/.local/share/virtualenvs/g-week1-repository-9jUi0dyS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

"""
7월 15일 도로교통법.pdf PyPDFLoader/RecursiveCharacterTextSplitter 사용하여 청킹 후
chromaDB 서버 rag_exam 컬렉션에 저장 완료
"""

# PDF 로드 및 청킹 후 임베딩하여 벡터 디비 저장
pdf_path = "docs/도로교통법.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# 메타데이터 설정
for page in pages:
    # 텍스트 전처리 적용
    page.page_content = clean_text(page.page_content)

# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기
    chunk_overlap=100,      # 청크 간 중복
    length_function=len,
    separators=[".", "\n\n"] # 분할 기준
)

# 전체 텍스트를 하나로 합쳐서 분할
full_text = '\n'.join([page.page_content for page in pages])
chunks = text_splitter.split_text(full_text)

print(f"\n총 {len(chunks)}개의 청크로 분할되었습니다.")

for i,chunk in enumerate(chunks):
    embedding = embeddings_model.embed_query(chunk)
    print(embedding)

    collection.add(
        embeddings = [embedding],
        documents = [chunk],
        ids = [f"chunk_{i}"]
    )


총 151개의 청크로 분할되었습니다.
[-0.04254503920674324, 0.023575792089104652, 0.06431805342435837, 0.0040624612011015415, 0.03520437702536583, -0.04896179214119911, -0.04945870861411095, 0.04134750738739967, 0.008773363195359707, -0.051516879349946976, 0.061043646186590195, -0.03499872237443924, 0.11062312871217728, -0.015146389603614807, -0.01949416846036911, -0.04237827658653259, 0.015465928241610527, 0.08467035740613937, -0.03916821628808975, 0.043566394597291946, 0.07695582509040833, -0.024440066888928413, -0.03142611309885979, 0.0021737203933298588, -0.06725775450468063, -0.04741029441356659, -0.08834333717823029, 0.04566483572125435, -0.03478369861841202, 0.012458187527954578, -0.0330328643321991, -0.02592252381145954, -0.016464030370116234, -0.03890812397003174, -0.10277937352657318, -0.05399135500192642, -0.04420347139239311, 0.07276754826307297, -0.049939531832933426, 0.07405056059360504, -0.09473246335983276, -0.09729531407356262, 0.04780704155564308, -0.0786968544125557, 0.040985450148

In [ ]:
# 사용자 질문 반환 함수
def get_user_query():
    lines = []

    while True:
        line = input("질문을 입력하세요 (종료: 빈 줄에서 Enter)")
        # 빈 줄이고 이전에 입력된 내용이 있으면 종료
        if not line and lines:
            break
        # 입력된 줄이 있으면 추가
        if line:
            lines.append(line)

    return "\n".join(lines)

In [ ]:
user_query = get_user_query()

user_query_embedding = embeddings_model.embed_query(user_query)
k = 2

results = collection.query(
    query_embeddings=[user_query_embedding],
    n_results=k,
    include=['documents']
)
print(user_query)

for i, chunk_id in enumerate(results['ids'][0]):
    print(f"{i}번째 청크 ID: {chunk_id}")
    print(f"해당 청크 내용: {results['documents'][0][i]}")  # 내용 앞부분만
    print('-------\n')


도로교통법 정리
0번째 청크 ID: chunk_109
해당 청크 내용: .<개정 2013. 3. 23., 2014. 11. 19., 2017. 7. 26.> 거짓이나 그 밖의 부정한 방법으로 강사자격증을 발급받은 경우 「교통사고처리 특례법」 제3조제1항 또는 「특정범죄 가중처벌 등에 관한 법률」 제5조의3을 위반하여 금고 이상의 형(집행유예를 포함한다)을 선고받은 경우 강사의 자격정지 기간 중에 교육을 한 경우 강사의 자격증을 다른 사람에게 빌려 준 경우 기능교육에 사용되는 자동차를 운전할 수 있는 운전면허가 취소된 경우 기능교육에 사용되는 자동차를 운전할 수 있는 운전면허의 효력이 정지된 경우 강사의 업무에 관하여 부정한 행위를 한 경우 제116조를 위반하여 대가를 받고 자동차운전교육을 한 경우 그 밖에 이 법이나 이 법에 따른 명령 또는 처분을 위반한 경우 ⑤ 전문학원의 학감은 강사가 아닌 사람으로 하여금 자동차운전에 관한 학과교육 또는 기능교육을 하게 하여서는 아니 된다. [전문개정 2011. 6. 8.] 제107조(기능검정원) ① 기능검정원이 되려는 사람은 행정안전부령으로 정하는 기능검정원 자격시험에 합격하고 경찰 청장이 지정하는 전문기관에서 자동차운전 기능검정에 관한 연수교육을 수료하여야 한다. <개정 2013. 3. 23., 11. 19., 2017. 7. 26.> ② 경찰청장은 제1항에 따른 연수교육을 수료한 사람에게 행정안전부령으로 정하는 바에 따라 기능검정원 자격증 을 발급하여야 한다.<개정 2013. 3. 23., 2014. 11. 19., 2017. 7. 26.> ③ 다음 각 호의 어느 하나에 해당하는 사람은 기능검정원이 될 수 없다
-------

1번째 청크 ID: chunk_0
해당 청크 내용: [시행 2018. 4. 25.] [법률 제15530호, 2018. 3. 27., 일부개정] 경찰청 (교통기획계(법제총괄))경찰청 (교통운영계(신호, 안전표지, 도로공사신고))경찰청 (운전면허계(운전면허))경찰청 (교통안전계(안전, 단속, 어린이

In [ ]:
import httpx


contexts = []

for i, doc in enumerate(results['documents'][0]):
    contexts.append(f"{i + 1}번째 관련 문서 : \n{doc}\n")

context_text = "\n".join(contexts)

# 프롬프트 생성
prompt = f"""질문: {user_query}
참고할 도로교통법 내용 :
{context_text}

위 도로교통법 내용을 바탕으로 질문에 답변해주시기 바랍니다.
"""
api_key = UPSTAGE_KEY_TM
base_url = "https://api.upstage.ai/v1/chat/completions"
payload = {
    "model": "solar-pro",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# timeout -> context가 길어 5초로는 부족, 60초로 늘려줌
with httpx.Client(timeout=60.0) as client:
    response = client.post(base_url, headers=headers, json=payload)

# 응답 결과 출력
print(response.status_code)
print(response.json())

200
{'id': '577043cb-9f9a-4504-9626-b69f656dfc34', 'object': 'chat.completion', 'created': 1752568775, 'model': 'solar-pro-250422', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '도로교통법은 도로에서 발생하는 교통상의 위험과 장애를 방지하고 제거하여 안전하고 원활한 교통을 보장하는 것을 목적으로 하며, 이를 위해 다양한 정의와 규정을 포함하고 있습니다.\n\n주요 내용은 다음과 같습니다:\n\n1. **도로의 정의**: \n   - 도로, 유료도로, 농어촌도로, 그리고 공개된 장소로서 교통의 안전과 원활성을 확보해야 하는 장소를 포함합니다.\n   - 자동차전용도로와 고속도로는 각각 자동차만 다닐 수 있는 도로와 고속 운행 전용 도로로 정의됩니다.\n\n2. **도로 구성 요소**:\n   - 차도, 중앙선, 차로의 정의가 포함되어 있으며, 이는 도로의 구조와 차마의 통행 방향을 명확히 하기 위함입니다.\n\n3. **강사 및 기능검정원의 자격 및 제한**:\n   - 강사자격증을 거짓이나 부정한 방법으로 발급받거나 금고 이상의 형을 선고받은 경우 자격이 취소될 수 있습니다.\n   - 강사는 자격정지 기간 동안 교육을 할 수 없으며, 자격증을 다른 사람에게 빌려 주는 것도 금지됩니다.\n   - 기능검정원이 되기 위해서는 특정 자격시험에 합격하고, 경찰청장이 지정한 전문기관에서 연수교육을 수료해야 합니다.\n   - 기능검정원이 될 수 없는 사유로는 특정 범죄를 저지르거나, 자격이 취소된 경우 등이 포함됩니다.\n\n4. **전문학원의 학감 의무**:\n   - 전문학원의 학감은 강사가 아닌 사람에게 자동차운전에 관한 교육을 시키지 않아야 합니다.\n\n이러한 규정들은 도로교통의 안전성을 확보하고, 교통 참여자들의 책임을 명확히 하기 위해 마련되었습니다.'}, 'logprobs'

In [ ]:
data = response.json()

print(data['choices'][0]['message']['content'])

도로교통법은 도로에서의 교통 안전을 보장하고 교통 상의 위험과 장해를 방지하며 원활한 교통을 확보하는 것을 목적으로 합니다. 주요 내용은 다음과 같습니다:

1. **도로의 정의**:
   - 도로법, 유료도로법, 농어촌도로 정비법에 따른 도로 및 현실적으로 불특정 다수의 사람 또는 차가 통행할 수 있는 공개된 장소를 포함합니다.
   - 자동차전용도로와 고속도로의 정의도 포함됩니다.

2. **도로의 구조 및 안전장치**:
   - 차도, 중앙선, 차로 등의 정의와 구분을 명시합니다.
   - 안전한 통행을 위해 필요한 표시 및 시설물에 대한 규정을 포함합니다.

3. **운전면허 및 교육**:
   - 운전면허의 취득 및 유지 조건, 운전면허의 취소 및 정지 사유 등이 명시됩니다.
   - 운전교육의 규정과 강사의 자격 요건 및 금지사항이 포함됩니다.

4. **교통법규 위반 및 처벌**:
   - 교통법규 위반 시 부과되는 벌금 및 징역형 등의 처벌 규정이 있습니다.
   - 교통사고처리 특례법 및 특정범죄 가중처벌 등에 관한 법률 등의 적용 사례가 포함됩니다.

5. **기능검정원의 자격 및 역할**:
   - 기능검정원의 자격 요건과 발급 절차, 기능검정원이 될 수 없는 경우 등이 규정됩니다.

이러한 내용은 교통 안전을 유지하고 교통법규를 준수하도록 하는 데 필수적입니다. 도로교통법은 지속적으로 개정되어 현대 사회의 변화하는 교통 환경에 적응하고 있습니다.
